# Spin Test 3

In [13]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import medfilt
import ast

Import video

In [14]:
video_number = "3"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = project_root / "data" / f"recording_{video_number}" / f"Recording_{video_number}.mp4" 
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

Opened: True, FPS: 59.94005994005994, Total Frames: 227.0


Import data

In [15]:
# Define the path to the CSV file
# input_data_path = project_root / "notebook" / "ball_detection" / "intermediate_data" / f"Circle_positions_cleaned_{video_number}.csv"
input_data_path = project_root / "data" / "auxiliary_data" / "circle_positions" / f"Adjusted_positions_new_{video_number}.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(input_data_path)

In [ ]:
# --- prepare writer ---
fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_path = str(project_root / "data" / f"recording_{video_number}" / "test_video" / f"Spin_test_{video_number}.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(str(out_path), fourcc, fps, (W, H))

# Prepare list to log axis endpoints
log = []

# Read frame0 and frame1
ret_curr, curr = cap.read()
ret_next, next_ = cap.read()
frame_idx = 0

while ret_curr and ret_next:
    vis = curr.copy()
    pA, pB = (np.nan, np.nan), (np.nan, np.nan)

    # Only compute if we have valid circle data
    if frame_idx < len(df) and not df.iloc[frame_idx][['x','y','radius']].isna().any():
        x, y, r = df.iloc[frame_idx][['x','y','radius']].astype(int)
        center = np.array([x, y])
        off = 2
        x0, x1 = max(x-r-off,0), min(x+r+off, W)
        y0, y1 = max(y-r-off,0), min(y+r+off, H)

        roi1 = curr[y0:y1, x0:x1]
        roi2 = next_[y0:y1, x0:x1]
        g1 = cv2.cvtColor(roi1, cv2.COLOR_BGR2GRAY)
        g2 = cv2.cvtColor(roi2, cv2.COLOR_BGR2GRAY)
        c_roi = np.array([x - x0, y - y0])

        axes = []
        for _ in range(3):            
            # Randomly sample points within the circle
            pts = []
            while len(pts) < 1000:
                xi = np.random.randint(0, g1.shape[1])
                yi = np.random.randint(0, g1.shape[0])
                if (xi-c_roi[0])**2 + (yi-c_roi[1])**2 <= (r-2)**2:
                    pts.append([xi, yi])
            p0 = np.array(pts, dtype=np.float32).reshape(-1,1,2)

            if p0.size == 0:
                continue

            p1, st, _ = cv2.calcOpticalFlowPyrLK(
                g1, g2, p0, None,
                winSize=(15,15), maxLevel=2,
                criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT, 10, 0.03)
            )

            old3d, new3d = [], []
            for o, n, s in zip(p0.reshape(-1,2), p1.reshape(-1,2), st.reshape(-1)):
                if s:
                    ox, oy = o - c_roi
                    nx, ny = n - c_roi
                    oz = np.sqrt(max(r*r - ox*ox - oy*oy, 0))
                    nz = np.sqrt(max(r*r - nx*nx - ny*ny, 0))
                    old3d.append([ox, oy, oz])
                    new3d.append([nx, ny, nz])

            if len(old3d) < 3:
                continue

            old3d = np.array(old3d)
            new3d = np.array(new3d)

            # Compute Rotation matrix using SVD
            Hmat = old3d.T @ new3d
            U, S, Vt = np.linalg.svd(Hmat)
            R = Vt.T @ U.T
            if np.linalg.det(R) < 0:
                Vt[-1,:] *= -1
                R = Vt.T @ U.T
            
            
            

            theta = np.arccos(np.clip((np.trace(R)-1)/2, -1,1))
            if np.sin(theta) != 0:
                ax = np.array([R[2,1]-R[1,2],
                               R[0,2]-R[2,0],
                               R[1,0]-R[0,1]])
                ax /= (2*np.sin(theta))
                ax /= np.linalg.norm(ax)
                axes.append(ax)
                

        if axes:
            avg = np.mean(np.vstack(axes), axis=0)
            avg /= np.linalg.norm(avg)

            pA_ball = avg * r
            pB_ball = -avg * r

            pA = tuple((center + pA_ball[:2]).astype(int))
            pB = tuple((center + pB_ball[:2]).astype(int))

            cv2.line(vis, pA, pB, (0,255,255), 2)
            cv2.circle(vis, pA, 2, (0,0,255), 2)
            cv2.circle(vis, pB, 2, (0,255,0), 2)


    # log the endpoints for this frame
    log.append({
        'frame': frame_idx,
        'pA_x': pA[0], 'pA_y': pA[1],
        'pB_x': pB[0], 'pB_y': pB[1]
    })

    out.write(vis)

    # advance frames
    frame_idx += 1
    curr = next_
    ret_next, next_ = cap.read()
    ret_curr = curr is not None

# write last frame if exists
if ret_curr:
    vis = curr.copy()
    log.append({'frame': frame_idx,
                'pA_x': np.nan, 'pA_y': np.nan,
                'pB_x': np.nan, 'pB_y': np.nan})
    out.write(vis)

cap.release()
out.release()

# Save the log to CSV
log_df = pd.DataFrame(log)
csv_path = str(project_root / "notebook" / "spin" / "intermediate_data" / f"Spin_data_mic_{video_number}.csv")
# log_df.to_csv(csv_path, index=False)

print(f"Video saved to {out_path} \nAxis log saved to {csv_path}")

# Add rotation center columns to ball_df
rotation_centers = []
for entry in log:
    if entry['pA_x'] != np.nan and entry['pA_y'] != np.nan and entry['pB_x'] != np.nan and entry['pB_y'] != np.nan:
        pA_z = np.sqrt(max(r*r - (entry['pA_x'] - x)**2 - (entry['pA_y'] - y)**2, 0))
        pB_z = np.sqrt(max(r*r - (entry['pB_x'] - x)**2 - (entry['pB_y'] - y)**2, 0))
        if pA_z < 0:
            rotation_centers.append({'rotation_center_x': entry['pA_x'], 'rotation_center_y': entry['pA_y']})
        else:
            rotation_centers.append({'rotation_center_x': entry['pB_x'], 'rotation_center_y': entry['pB_y']})
    else:
        rotation_centers.append({'rotation_center_x': np.nan, 'rotation_center_y': np.nan})

rotation_centers_df = pd.DataFrame(rotation_centers)
ball_df = pd.concat([df, rotation_centers_df], axis=1)
ball_df.to_csv(csv_path, index=False)

Frame 37: Found 7 points to track
Frame 37: Found 7 points to track
Frame 37: Found 7 points to track
Frame 38: Found 4 points to track
Frame 38: Found 4 points to track
Frame 38: Found 4 points to track
Frame 39: Found 9 points to track
Frame 39: Found 9 points to track
Frame 39: Found 9 points to track
Frame 40: Found 9 points to track
Frame 40: Found 9 points to track
Frame 40: Found 9 points to track
Frame 41: Found 4 points to track
Frame 41: Found 4 points to track
Frame 41: Found 4 points to track
Frame 42: Found 5 points to track
Frame 42: Found 5 points to track
Frame 42: Found 5 points to track
Frame 43: Found 7 points to track
Frame 43: Found 7 points to track
Frame 43: Found 7 points to track
Frame 44: Found 5 points to track
Frame 44: Found 5 points to track
Frame 44: Found 5 points to track
Frame 45: Found 4 points to track
Frame 45: Found 4 points to track
Frame 45: Found 4 points to track
Frame 46: Found 7 points to track
Frame 46: Found 7 points to track
Frame 46: Foun